In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def load_images_from_folder(folder, image_size=(128, 128)):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                img_path = os.path.join(label_folder, filename)
                try:
                    img = Image.open(img_path).convert('RGB')
                    img = img.resize(image_size)
                    img_array = np.array(img).flatten()  # Flatten the image
                    images.append(img_array)
                    labels.append(label)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)

In [4]:
def count_classes(folder_path):
    return len([name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))])

In [6]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return accuracy, precision, recall, f1

In [7]:
train_folder_path = '/content/drive/MyDrive/Dataset/Train'
test_folder_path = '/content/drive/MyDrive/Dataset/Test'
X_train, y_train = load_images_from_folder(train_folder_path, image_size=(128, 128))
X_test, y_test = load_images_from_folder(test_folder_path, image_size=(128, 128))

In [11]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)



# Standardize the data (recommended for certain algorithms)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Define models
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier(n_neighbors=1),
    "Logistic Regression": LogisticRegression(max_iter=500),  # Increased max_iter
    "SVM": SVC(),
    "ANN": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=100)  # Adjust ANN parameters
}

# Evaluate each model
results = []

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train_encoded)  # Use scaled data
    y_pred = model.predict(X_test_scaled)  # Ensure predictions are class labels
    accuracy = accuracy_score(y_test_encoded, y_pred)
    precision = precision_score(y_test_encoded, y_pred, average='weighted')
    recall = recall_score(y_test_encoded, y_pred, average='weighted')
    f1 = f1_score(y_test_encoded, y_pred, average='weighted')
    results.append([model_name, accuracy, precision, recall, f1])



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
 # Convert the metrics to percentages
results_percentage = []
for result in results:
    algorithm = result[0]
    accuracy = result[1] * 100
    precision = result[2] * 100
    recall = result[3] * 100
    f1_score = result[4] * 100
    results_percentage.append([algorithm, accuracy, precision, recall, f1_score])

# Display results in a table format
import pandas as pd
results_df = pd.DataFrame(results_percentage, columns=["Algorithm", "Accuracy (%)", "Precision (%)", "Recall (%)", "F1 Score (%)"])
print(results_df)

             Algorithm  Accuracy (%)  Precision (%)  Recall (%)  F1 Score (%)
0        Decision Tree        55.000      54.456657      55.000     54.086533
1                  KNN        58.125      57.792534      58.125     56.428478
2  Logistic Regression        66.250      66.200660      66.250     65.782412
3                  SVM        72.500      74.068182      72.500     71.469346
4                  ANN        61.875      62.325911      61.875     61.223795
